# Zero to Snowflake - Horizonによるガバナンス

## トピック
1. ロールとアクセス制御の紹介
2. 自動タグ付けによるタグベース分類
3. マスキングポリシーによる列レベルセキュリティ
4. 行アクセスポリシーによる行レベルセキュリティ
5. データメトリック関数によるデータ品質監視
6. Trust Centerによるアカウントセキュリティ監視

## セッション設定

In [ ]:
%%sql
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "governance_with_horizon"}}';

In [ ]:
%%sql
USE ROLE useradmin;
USE DATABASE tb_101;
USE WAREHOUSE tb_dev_wh;

## 1. ロールとアクセス制御

In [ ]:
%%sql
-- 既存のロールを確認
SHOW ROLES;

In [ ]:
%%sql
-- データスチュワードロールを作成
CREATE OR REPLACE ROLE tb_data_steward
    COMMENT = 'カスタムロール';

In [ ]:
%%sql
USE ROLE securityadmin;

-- ウェアハウス権限を付与
GRANT OPERATE, USAGE ON WAREHOUSE tb_dev_wh TO ROLE tb_data_steward;

-- データベースとスキーマ権限を付与
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_data_steward;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_data_steward;

-- テーブル権限を付与
GRANT SELECT ON ALL TABLES IN SCHEMA raw_customer TO ROLE tb_data_steward;
GRANT ALL ON SCHEMA governance TO ROLE tb_data_steward;
GRANT ALL ON ALL TABLES IN SCHEMA governance TO ROLE tb_data_steward;

In [ ]:
%%sql
-- 現在のユーザーにロールを付与
SET my_user = CURRENT_USER();
GRANT ROLE tb_data_steward TO USER IDENTIFIER($my_user);

In [ ]:
%%sql
USE ROLE tb_data_steward;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 2. 自動タグ付けによるタグベース分類

In [ ]:
%%sql
USE ROLE accountadmin;

-- PIIタグを作成
CREATE OR REPLACE TAG governance.pii;
GRANT APPLY TAG ON ACCOUNT TO ROLE tb_data_steward;

-- 分類実行権限を付与
GRANT EXECUTE AUTO CLASSIFICATION ON SCHEMA raw_customer TO ROLE tb_data_steward;
GRANT DATABASE ROLE SNOWFLAKE.CLASSIFICATION_ADMIN TO ROLE tb_data_steward;
GRANT CREATE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE ON SCHEMA governance TO ROLE tb_data_steward;

In [ ]:
%%sql
USE ROLE tb_data_steward;

-- 分類プロファイルを作成
CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE
  governance.tb_classification_profile(
    {
      'minimum_object_age_for_classification_days': 0,
      'maximum_classification_validity_days': 30,
      'auto_tag': true
    });

In [ ]:
%%sql
-- タグマップを設定
CALL governance.tb_classification_profile!SET_TAG_MAP(
  {'column_tag_map':[
    {
      'tag_name':'tb_101.governance.pii',
      'tag_value':'pii',
      'semantic_categories':['NAME', 'PHONE_NUMBER', 'POSTAL_CODE', 'DATE_OF_BIRTH', 'CITY', 'EMAIL']
    }]});

In [ ]:
%%sql
-- テーブルを自動分類
CALL SYSTEM$CLASSIFY('tb_101.raw_customer.customer_loyalty', 'tb_101.governance.tb_classification_profile');

In [ ]:
%%sql
-- タグ付けの結果を確認
SELECT 
    column_name,
    tag_database,
    tag_schema,
    tag_name,
    tag_value,
    apply_method
FROM TABLE(INFORMATION_SCHEMA.TAG_REFERENCES_ALL_COLUMNS('raw_customer.customer_loyalty', 'table'));

## 3. マスキングポリシーによる列レベルセキュリティ

In [ ]:
%%sql
-- 機密文字列データ用のマスキングポリシー
CREATE OR REPLACE MASKING POLICY governance.mask_string_pii AS (original_value STRING)
RETURNS STRING ->
  CASE WHEN
    CURRENT_ROLE() NOT IN ('ACCOUNTADMIN', 'TB_ADMIN')
    THEN '****MASKED****'
    ELSE original_value
  END;

In [ ]:
%%sql
-- 機密DATEデータ用のマスキングポリシー
CREATE OR REPLACE MASKING POLICY governance.mask_date_pii AS (original_value DATE)
RETURNS DATE ->
  CASE WHEN
    CURRENT_ROLE() NOT IN ('ACCOUNTADMIN', 'TB_ADMIN')
    THEN DATE_TRUNC('year', original_value)
    ELSE original_value
  END;

In [ ]:
%%sql
-- タグにマスキングポリシーを適用
ALTER TAG governance.pii SET
    MASKING POLICY governance.mask_string_pii,
    MASKING POLICY governance.mask_date_pii;

In [ ]:
%%sql
-- publicロールでマスキングを確認
USE ROLE public;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

In [ ]:
%%sql
-- 管理者ロールでマスキングが適用されないことを確認
USE ROLE tb_admin;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 4. 行アクセスポリシーによる行レベルセキュリティ

In [ ]:
%%sql
USE ROLE tb_data_steward;

-- 行ポリシーマップを作成
CREATE OR REPLACE TABLE governance.row_policy_map
    (role STRING, country_permission STRING);

INSERT INTO governance.row_policy_map
    VALUES('tb_data_engineer', 'United States');

In [ ]:
%%sql
-- 行アクセスポリシーを作成
CREATE OR REPLACE ROW ACCESS POLICY governance.customer_loyalty_policy
    AS (country STRING) RETURNS BOOLEAN ->
        CURRENT_ROLE() IN ('ACCOUNTADMIN', 'SYSADMIN') 
        OR EXISTS 
            (
            SELECT 1
                FROM governance.row_policy_map rp
            WHERE
                UPPER(rp.role) = CURRENT_ROLE()
                AND rp.country_permission = country
            );

In [ ]:
%%sql
-- テーブルに行アクセスポリシーを適用
ALTER TABLE raw_customer.customer_loyalty
    ADD ROW ACCESS POLICY governance.customer_loyalty_policy ON (country);

In [ ]:
%%sql
-- tb_data_engineerロールで行フィルタリングを確認
USE ROLE tb_data_engineer;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 5. データメトリック関数によるデータ品質監視

In [ ]:
%%sql
USE ROLE tb_data_steward;

-- null値の割合をチェック
SELECT SNOWFLAKE.CORE.NULL_PERCENT(SELECT customer_id FROM raw_pos.order_header);

In [ ]:
%%sql
-- 重複をチェック
SELECT SNOWFLAKE.CORE.DUPLICATE_COUNT(SELECT order_id FROM raw_pos.order_header);

In [ ]:
%%sql
-- 平均値を計算
SELECT SNOWFLAKE.CORE.AVG(SELECT order_total FROM raw_pos.order_header);

In [ ]:
%%sql
-- カスタムデータメトリック関数を作成
CREATE OR REPLACE DATA METRIC FUNCTION governance.invalid_order_total_count(
    order_prices_t table(
        order_total NUMBER,
        unit_price NUMBER,
        quantity INTEGER
    )
)
RETURNS NUMBER
AS
'SELECT COUNT(*)
 FROM order_prices_t
 WHERE order_total != unit_price * quantity';

## 6. Trust Center

Trust Centerは、Snowsight UIの「監視」→「Trust Center」からアクセスできます。

Trust Centerでできること：
- セキュリティスキャナーの有効化
- セキュリティ違反の検出
- リスク評価
- 修復アクションの実行

In [ ]:
%%sql
USE ROLE accountadmin;
GRANT APPLICATION ROLE SNOWFLAKE.TRUST_CENTER_ADMIN TO ROLE tb_admin;

## クリーンアップ

In [ ]:
%%sql
USE ROLE accountadmin;
DROP ROLE IF EXISTS tb_data_steward;
ALTER TAG IF EXISTS governance.pii UNSET MASKING POLICY governance.mask_string_pii, MASKING POLICY governance.mask_date_pii;
DROP MASKING POLICY IF EXISTS governance.mask_string_pii;
DROP MASKING POLICY IF EXISTS governance.mask_date_pii;
ALTER TABLE raw_customer.customer_loyalty DROP ROW ACCESS POLICY governance.customer_loyalty_policy;
DROP ROW ACCESS POLICY IF EXISTS governance.customer_loyalty_policy;
DROP TAG IF EXISTS governance.pii;
ALTER SESSION UNSET query_tag;
ALTER WAREHOUSE tb_dev_wh SUSPEND;